Convelution Neural Network

In [16]:
import torch
import torch.nn as nn
import pandas as pd

In [17]:
df = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [18]:
X = df.iloc[:, 1:].values
Y = df.iloc[:, 0].values
#X_test = df.iloc[:, 1:].values 
#Y_test = df.iloc[:, 1:].values

In [19]:
#reshape the training data
X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1, 28, 28) / 255.0
Y = torch.tensor(Y, dtype=torch.long)
#X_test = torch.tensor(X_test, dtype=torch.float32).reshape(-1, 1, 28, 28)/ 255.0
#Y_test = torch.tensor(Y_test, dtype=torch.long)
                                                    

In [20]:
#the neural network model
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
    nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
    nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 10),
    # we end with 10 node for the ten possible digits
    # pytorch automatically uses a softmax activation function here where all probabilities of the ten nodes add up to 1.0
)

In [27]:
epochs = 1
batch_size = 100
batches_per_epoch = len(X) // batch_size

optimizer = torch.optim.Adam(model.parameters(), lr=.01)
loss_fn = nn.CrossEntropyLoss()
    

In [29]:
for epoch in range(epochs):

    for i in range(batches_per_epoch):

        #get a batch
        start = i * batch_size
        X_batch = X[start:start+batch_size]
        Y_batch = Y[start:start+batch_size]

        #forward pass
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)

        #backward pass
        optimizer.zero_grad()
        loss.backward()

        #update weights
        optimizer.step()
        
    #Validation
   
    y_pred = model(X)
    acc = (torch.argmax(y_pred, 1) == Y).float().mean()
    
    print(f"Epoch {epoch}: model accuracy {acc}")
        

Epoch 0: model accuracy 0.9937857389450073


In [26]:
#Needed to move to testing mode
import torch
import csv
import pandas as pd  # Added pandas import
import numpy as np

model.eval()

#Create list to store predictions
ImageId = []
predictions = []

# Convert DataFrame to tensor if X_test is a DataFrame
if isinstance(X_test, pd.DataFrame):


    X_test_np = X_test.values.astype(np.float32)
    X_test_np = X_test_np.reshape(-1, 1, 28, 28)
    X_test = torch.tensor(X_test_np, dtype=torch.float32)
    
    #X_test = torch.tensor(X_test.values, dtype=torch.float32).reshape(-1, 1, 28, 28)/ 255.0
  
# If X_test is a numpy array, convert to tensor
#elif isinstance(X_test, np.ndarray):
    #X_test = torch.tensor(X_test, dtype=torch.float32)

#Disable gradient calculations
# Create a list to store predictions
with torch.no_grad():
    # Forward pass on test data
    test_outputs = model(X_test)
    _, test_predictions = torch.max(test_outputs.data, 1)
    
    # Convert to numpy arrays for use in the csv submissions
    predictions = test_predictions.cpu().numpy()
    # Create item IDs 
    ImageId = list(range(1,len(predictions)+1))

# Create DataFrame with predictions
results_df = pd.DataFrame({
    'ImageId': ImageId,
    'label': predictions
})

# Save to CSV
results_df.to_csv('ac_cnn_predictions6_10-23-25.csv', index=False)
print("Predictions saved to ac_cnn_predictions6_10-23-25.csv")


Predictions saved to ac_cnn_predictions6_10-23-25.csv
